In [1]:
from sequential_tasks import EchoData
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import SimpleRNN, Dense, TimeDistributed
import numpy as np


In [2]:
def echo_experiment(settings):
    train_data_gen = EchoData(
        series_length=settings['series_length'],
        truncated_length=settings['truncated_length'],
        echo_step=settings['echo_step'],
        batch_size=settings['batch_size'])

    model = Sequential([
        SimpleRNN(
            units=settings['h_units'],
            batch_input_shape=(settings['batch_size'], settings['truncated_length'], 1),
            return_sequences=True,
            stateful=True),
        TimeDistributed(
            Dense(units=1, activation='sigmoid'))
    ])

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit_generator(
        generator=train_data_gen,
        epochs=settings['max_epochs'],
        verbose=2,
        shuffle=False)

    # testing
    test_data_gen = EchoData(
        series_length=settings['series_length'],
        truncated_length=settings['truncated_length'],
        echo_step=settings['echo_step'],
        batch_size=settings['batch_size'])

    # we could do evaluation like this:
    # eval_metrics = model.evaluate_generator(test_data_gen)
    
    # but let's gather statistics from each batch...
    batch_accuracies = []
    for b in range(test_data_gen.n_batches):
        x_test, y_test = test_data_gen[b]
        batch_metrics = model.evaluate(
            x=x_test,
            y=y_test,
            batch_size=params["batch_size"],
            verbose=0)
        batch_accuracies.append(100. * batch_metrics[1])
    # ... and let's skip the first batch (when RNN is not warmed up)
    test_accuracy = np.mean(batch_accuracies[1:])

    return test_accuracy


In [3]:
# experiments settings
params = {
    "series_length": 20000,
    "echo_step": 3,
    "truncated_length": 20,
    "batch_size": 5,
    "h_units": 4,
    "max_epochs": 4
}

acc = echo_experiment(params)
print('acc = {:.2f}%.'.format(acc))


Epoch 1/4
 - 7s - loss: 0.6507 - acc: 0.5941
Epoch 2/4
 - 6s - loss: 0.3372 - acc: 0.8691
Epoch 3/4
 - 6s - loss: 0.1511 - acc: 0.9244
Epoch 4/4
 - 6s - loss: 0.1228 - acc: 0.9256
acc = 100.00%.
